In [10]:
!pip install numpy
!pip install tensorflow
!pip install matplotlib
!pip install scikit-learn
!pip install keras-tuner
!pip install pandas



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import re
import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import re
import os
import re
import shutil
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, LayerNormalization, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
import os

In [12]:
image_path='image'
def copy_files_by_pattern(directory_path, destination_path, regex_pattern):
    """
    Copies files from the source directory to the destination directory based on a regex pattern.

    Args:
    directory_path (str): Path to the directory containing the files.
    destination_path (str): Directory where the copies will be stored.
    regex_pattern (str): Regular expression pattern to match filenames.

    Returns:
    None
    """
    # Compile the regex pattern
    pattern = re.compile(regex_pattern)

    # Create the destination folder if it doesn't exist
    os.makedirs(destination_path, exist_ok=True)

    # Initialize a counter for copied files
    copied_files_count = 0

    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the filename matches the pattern
        if pattern.match(filename):
            # Path of the current file
            current_file_path = os.path.join(directory_path, filename)
            # Path where the file will be copied
            destination_file_path = os.path.join(destination_path, filename)
            # Copy the file
            shutil.copy2(current_file_path, destination_file_path)
            copied_files_count += 1
            print(f'Copied: {filename}')

    # Print the result
    print(f'All matching files have been copied to {destination_path}. Total files copied: {copied_files_count}')


cell_types = [
    "spherocyte",
    "echinocyte",
    "keratocytes",
    "multilobate_cells",
    "cell_clusters",
    "knizocytes",
    "multilobate_cells",
    "acanthocytes",
    "stomatocyte"
]

for cell_type in cell_types:
    directory_path = image_path
    destination_path = image_path + '/' + cell_type
    regex_pattern = rf'.*{cell_type}.*\.tif$'

    copy_files_by_pattern(directory_path, destination_path, regex_pattern)


Copied: -1.00_spherocyte000129.tif
Copied: -1.00_spherocyte000130.tif
Copied: -1.00_spherocyte000116.tif
Copied: -1.00_spherocyte000113.tif
Copied: -1.00_spherocyte000134.tif
Copied: -1.00_spherocyte000132.tif
Copied: -1.00_spherocyte000127.tif
Copied: -1.00_spherocyte000118.tif
Copied: -1.00_spherocyte000133.tif
Copied: -1.00_spherocyte000135.tif
Copied: -1.00_spherocyte000122.tif
Copied: -1.00_spherocyte000131.tif
Copied: -1.00_spherocyte000128.tif
Copied: -1.00_spherocyte000124.tif
Copied: -1.00_spherocyte000125.tif
Copied: -1.00_spherocyte000126.tif
Copied: -1.00_spherocyte000123.tif
Copied: -1.00_spherocyte000121.tif
Copied: -1.00_spherocyte000120.tif
Copied: -1.00_spherocyte000119.tif
Copied: -1.00_spherocyte000117.tif
Copied: -1.00_spherocyte000112.tif
Copied: -1.00_spherocyte000114.tif
Copied: -1.00_spherocyte000115.tif
Copied: -1.00_spherocyte000110.tif
Copied: -1.00_spherocyte000109.tif
Copied: -1.00_spherocyte000075.tif
Copied: -1.00_spherocyte000052.tif
Copied: -1.00_sphero

In [13]:
def create_patches(inputs, patch_size, num_patches, projection_dim, return_patches=False):
    patches = layers.Conv2D(filters=projection_dim, kernel_size=patch_size, strides=patch_size, padding="valid")(inputs)
    reshaped_patches = layers.Reshape((num_patches, projection_dim))(patches)
    if return_patches:
        return patches  # Return patches for visualization
    return reshaped_patches

def add_position_embedding(patches, num_patches, projection_dim):
    position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)(tf.range(num_patches))
    return patches + position_embedding

def transformer_encoder(patches, projection_dim, num_heads, transformer_depth):
    for _ in range(transformer_depth):
        # Normalize the input patches
        x1 = layers.LayerNormalization(epsilon=1e-6)(patches)
        
        # Apply Multi-Head Attention
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        
        # Add the attention output back to the input (Skip Connection)
        x2 = layers.Add()([attention_output, patches])
        
        # Normalize the result of the addition
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # Apply a feed-forward network (FFN) within the transformer, first expanding then projecting back
        x3 = layers.Dense(units=projection_dim*2, activation="relu")(x3)
        patches = layers.Dense(units=projection_dim)(x3)
        
        # Add the output of the FFN back to the combined result of the attention output and the input (Second Skip Connection)
        patches = layers.Add()([patches, x2])
        
    return patches

def build_classifier(patches, num_classes):
    # Normalize the input
    representation = layers.LayerNormalization(epsilon=1e-6)(patches)
    
    # Flatten the normalized tensor to prepare for the dense layer
    representation = layers.Flatten()(representation)
    
    # Dropout for regularization
    features = layers.Dropout(0.5)(representation)
    
    # Dense layer for learning non-linear combinations
    features = layers.Dense(128, activation="relu")(features)
    
    # Another dropout for regularization
    features = layers.Dropout(0.5)(features)
    
    # Final dense layer with softmax activation for classification
    outputs = layers.Dense(num_classes, activation="softmax")(features)
    
    return outputs

def create_vit_classifier(input_shape, patch_size, num_patches, projection_dim, num_heads, transformer_depth, num_classes):
    # Define the input layer with the specified input shape
    inputs = layers.Input(shape=input_shape)
    
    # Extract patches from the input image and project to a higher-dimensional space
    patches = create_patches(inputs, patch_size, num_patches, projection_dim)
    
    # Add positional embeddings to the patches to retain positional information
    patches = add_position_embedding(patches, num_patches, projection_dim)
    
    # Pass the patches through the transformer encoder to process interactions between patches
    patches = transformer_encoder(patches, projection_dim, num_heads, transformer_depth)
    
    # Build the classifier head to make final predictions from the encoded features
    outputs = build_classifier(patches, num_classes)
    
    # Create the final Keras model with inputs as the image and outputs as the class predictions
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set paths to your data directories
image_path='image'

data_dir = image_path+'/Classes'


datagen = ImageDataGenerator(
    rescale=1./255,    # Normalize pixel values
    validation_split=0.2,  # Reserve 20% for validation
    rotation_range=20,  # Randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image 
    width_shift_range=0.1,  # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # Randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # Randomly flip images
    vertical_flip=False,  # Do not randomly flip images vertically
    shear_range=0.2,  # Shear intensity
    brightness_range=[0.2,1.0],  # Range for picking a brightness shift value
    fill_mode='nearest',  # Points outside the boundaries are filled according to the given mode
    cval=0.0  # Value used for points outside the boundaries when fill_mode = "constant"
)

# Setup the train generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),  # Load images at 64x64
    batch_size=32,
    class_mode='sparse',  # Ensure this matches the loss function
    subset='training',  # Set as training data
    shuffle=True
)

# Setup the validation generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),  # Load images at 64x64
    batch_size=32,
    class_mode='sparse',  # Ensure this matches the loss function
    subset='validation',  # Set as validation data
    shuffle=True
)


# Convert the generators to tf.data.Dataset
def generator_to_dataset(generator):
    return tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=(
            tf.TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )
    )

train_dataset = generator_to_dataset(train_generator).repeat()
validation_dataset = generator_to_dataset(validation_generator).repeat()


Found 523 images belonging to 8 classes.
Found 126 images belonging to 8 classes.


2024-05-20 04:09:02.942895: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46676 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:21:00.0, compute capability: 8.9


In [16]:
from sklearn.utils import class_weight
import numpy as np

# Get the class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights_dict)


Class weights: {0: 0.920774647887324, 1: 1.1674107142857142, 2: 0.2793803418803419, 3: 2.615, 4: 3.4407894736842106, 5: 6.5375, 6: 0.8716666666666667, 7: 1.981060606060606}


In [18]:
# Function to build the model for hyperparameter tuning
import numpy as np
import tensorflow as tf
from sklearn.utils import class_weight
from keras_tuner import RandomSearch
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback


# Function to build the model for hyperparameter tuning
def build_model(hp):
    num_layers = hp.Int('num_layers', min_value=2, max_value=50, step=1)
    num_heads = hp.Int('num_heads', min_value=4, max_value=50, step=2)
    d_model = hp.Int('d_model', min_value=64, max_value=456, step=64)
    transformer_depth = hp.Int('transformer_depth', min_value=2, max_value=6, step=1)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    
    model = create_vit_classifier(
        input_shape=(64, 64, 3),
        patch_size=8,
        num_patches=(64 // 8) ** 2,
        projection_dim=d_model,
        num_heads=num_heads,
        transformer_depth=transformer_depth,
        num_classes=8
    )
    
    lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=Adam(learning_rate=lr), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

# Custom callback to stop training if accuracy is below a threshold after a specific number of epochs
class EarlyStoppingByAccuracyAfterNEpochs(Callback):
    def __init__(self, monitor='accuracy', value=0.45, patience=7, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.patience = patience
        self.verbose = verbose
        self.epoch_count = 0

    def on_epoch_end(self, epoch, logs=None):
        self.epoch_count += 1
        logs = logs or {}
        acc = logs.get(self.monitor)
        if self.epoch_count >= self.patience:
            if acc is not None and acc < self.value:
                if self.verbose > 0:
                    print(f"Epoch {epoch}: early stopping threshold reached: {acc} < {self.value}")
                self.model.stop_training = True


# Initialize the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=80,
    executions_per_trial=1,
    directory='my_dir_new',
    project_name='hparam_tuning_new'
)

# Define the early stopping callback
early_stopping = EarlyStoppingByAccuracyAfterNEpochs(monitor='accuracy', value=0.45, verbose=1)


# Perform the hyperparameter search with class weights and the custom callback
tuner.search(train_generator, 
             epochs=10, 
             validation_data=validation_generator, 
             class_weight=class_weights_dict,
             callbacks=[early_stopping])

# Retrieve the best hyperparameters and print them
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

Trial 80 Complete [00h 00m 28s]
val_accuracy: 0.1428571492433548

Best val_accuracy So Far: 0.460317462682724
Total elapsed time: 01h 03m 06s


In [20]:
# Retrieve the best hyperparameters and print them
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. 
The optimal number of layers is {best_hps.get('num_layers')}.
The optimal number of heads is {best_hps.get('num_heads')}.
The optimal projection dimension is {best_hps.get('d_model')}.
The optimal transformer depth is {best_hps.get('transformer_depth')}.
The optimal dropout rate is {best_hps.get('dropout_rate')}.
The optimal learning rate is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. 
The optimal number of layers is 4.
The optimal number of heads is 16.
The optimal projection dimension is 384.
The optimal transformer depth is 2.
The optimal dropout rate is 0.5.
The optimal learning rate is 0.0003699594136815919.



In [22]:
trial_params = {
    "num_layers": 4,
    "num_heads": 16,
    "d_model": 384,
    "mlp_dim": 2,
    "dropout_rate": 0.5,
    "learning_rate": 0.0003699594136815919
}

In [23]:
# Model configuration
input_shape = (64, 64, 3)  # Assuming RGB images of size 64x64
patch_size = 4  # Patch size: 4x4
num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
projection_dim = trial_params["d_model"]
num_heads = trial_params["num_heads"]
transformer_depth = trial_params["num_layers"]
#mlp_dim = trial_params["mlp_dim"]
dropout_rate = trial_params["dropout_rate"]
learning_rate = trial_params["learning_rate"]
num_classes = len(train_generator.class_indices)  # Number of classes based on the generator

In [24]:
num_classes

8

In [25]:
model = create_vit_classifier(input_shape, patch_size, num_patches, projection_dim, num_heads, transformer_depth, num_classes)
model.summary()

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 16, 16, 384)          18816     ['input_2[0][0]']             
                                                                                                  
 reshape_1 (Reshape)         (None, 256, 384)             0         ['conv2d_1[0][0]']            
                                                                                                  
 tf.math.add_1 (TFOpLambda)  (None, 256, 384)             0         ['reshape_1[0][0]']           
                                                                                            

In [31]:
import scipy

history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=100,  # Increased number of epochs
                    steps_per_epoch=len(train_generator),
                    validation_steps=len(validation_generator))

Epoch 1/100
17/17 [==============================] - 3s 162ms/step - loss: 1.9147 - accuracy: 0.4455 - val_loss: 1.8870 - val_accuracy: 0.5079
Epoch 2/100
17/17 [==============================] - 3s 153ms/step - loss: 1.9128 - accuracy: 0.4570 - val_loss: 1.9083 - val_accuracy: 0.4524
Epoch 3/100
17/17 [==============================] - 3s 165ms/step - loss: 1.9059 - accuracy: 0.4532 - val_loss: 1.9022 - val_accuracy: 0.4603
Epoch 4/100
17/17 [==============================] - 3s 149ms/step - loss: 1.9167 - accuracy: 0.4264 - val_loss: 1.8989 - val_accuracy: 0.4603
Epoch 5/100
17/17 [==============================] - 3s 150ms/step - loss: 1.8980 - accuracy: 0.4685 - val_loss: 1.8798 - val_accuracy: 0.5000
Epoch 6/100
17/17 [==============================] - 3s 155ms/step - loss: 1.9092 - accuracy: 0.4302 - val_loss: 1.9127 - val_accuracy: 0.4365
Epoch 7/100
17/17 [==============================] - 3s 157ms/step - loss: 1.8916 - accuracy: 0.4532 - val_loss: 1.8910 - val_accuracy: 0.4603

In [32]:
model.save("vision_transformer_model_regularization_vitimageclassificationcopy.keras")

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

In [34]:
# Evaluate the model
validation_steps = len(validation_generator)
predictions = model.predict(validation_generator, steps=validation_steps)
predicted_classes = predictions.argmax(axis=1)
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

4/4 [==============================] - 1s 130ms/step
                   precision    recall  f1-score   support

     acanthocytes       0.00      0.00      0.00        17
    cell_clusters       0.00      0.00      0.00        13
       echinocyte       0.46      1.00      0.63        58
      keratocytes       0.00      0.00      0.00         6
       knizocytes       0.00      0.00      0.00         4
multilobate_cells       0.00      0.00      0.00         2
       spherocyte       0.00      0.00      0.00        18
      stomatocyte       0.00      0.00      0.00         8

         accuracy                           0.46       126
        macro avg       0.06      0.12      0.08       126
     weighted avg       0.21      0.46      0.29       126



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
# Check the distribution of classes in the training and validation sets
train_class_counts = np.bincount(train_generator.classes)
val_class_counts = np.bincount(validation_generator.classes)

print("Training class distribution:", train_class_counts)
print("Validation class distribution:", val_class_counts)


Training class distribution: [ 71  56 234  25  19  10  75  33]
Validation class distribution: [17 13 58  6  4  2 18  8]
